In [1]:
from webbot import Browser
from selenium import webdriver
import time
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
import csv
from datetime import datetime

## ETAPE 1 : SE CONNECTER A ATOM

In [2]:
def access_atom(driver):
    """
    Accède à la plateforme Phoebus en utilisant un objet de navigateur.

    Cette fonction effectue les étapes suivantes :
    1. Navigue vers la page d'accueil de Atom Archives.
    2. Clique sur le bouton "Ouverture de session".
    3. Utilise les identifiants stockés dans le fichier 'credentials.json' pour se connecter.

    Args:
        driver: Objet de navigateur permettant d'automatiser les interactions avec la page web.
    Note:
        Assurez-vous d'avoir un fichier 'credentials.json' contenant les champs 'email' et 'password'
        pour effectuer la connexion à Phoebus.
    """
        
    driver.go_to('https://atom-archives.unil.ch/index.php')
    driver.click('Ouverture de session', tag='button')
    
    with open('credentials.json', 'r') as jsonFile:
        credentials = json.load(jsonFile)
        
    driver.type(credentials['email'], id='email')
    driver.type(credentials['password'], id='password')
    driver.click('Ouverture de session')

# LANCEMENT DE LA FONCTION
driver = Browser() #Browser pour ouvrir un navigateur
access_atom(driver)

## ETAPE 2 : ACQUISITION DES DONNEES

In [3]:
def extract_autorite(csv_path):
    """
    Extrait les noms à partir du fichier CSV contenant les noms et les données récupérées.
    Args:
        csv_path (str): Chemin vers le fichier CSV contenant les données récupérées.

    Returns:
        autorities (list): Liste des données récupérées.
    """
    
    # Charger le fichier CSV
    csv_path = '01_noticesACompleter.csv'
    df = pd.read_csv(csv_path, na_values=['NULL'], delimiter=',') #delimiter dépend de la configuration du separator du csv
    

    # Extraire les noms de la colonne "autoritesPHOEBUS"
    autorities = df.values.tolist()

    return autorities
    
# LANCEMENT DE LA FONCTION
csv_path = '01_noticesACompleter.csv'
autorities = extract_autorite(csv_path)
# Afficher les noms (à titre de vérification)
print(autorities)

[['ABRIQUET Evelyne', 1, 'https://atom-archives.unil.ch/index.php/abriquet-evelyne', nan, nan, nan, nan], ['ABRIQUET Michèle', 1, 'https://atom-archives.unil.ch/index.php/abriquet-michele', nan, nan, nan, nan], ['ACHTONI Marc', 1, 'https://atom-archives.unil.ch/index.php/achtoni-marc', nan, nan, nan, nan], ['AEBISCHER Jean-Marie', 1, 'https://atom-archives.unil.ch/index.php/aebischer-jean-marie', nan, nan, nan, nan], ['AEGATA Isabelle', 1, 'https://atom-archives.unil.ch/index.php/aegata-isabelle', nan, nan, nan, nan], ['AGUET Pierre', 1, 'https://atom-archives.unil.ch/index.php/aguet-pierre', nan, nan, nan, nan], ['ALAMHAS Emmanuelle', 1, 'https://atom-archives.unil.ch/index.php/alamhas-emmanuelle', nan, nan, nan, nan], ['ALFF Wilhelm', 1, 'https://atom-archives.unil.ch/index.php/alff-wilhelm', nan, nan, nan, nan], ['ALLAZ Bernard', 1, 'https://atom-archives.unil.ch/index.php/allaz-bernard', nan, nan, nan, nan], ['ALTENDORF Wolfgang', 1, 'https://atom-archives.unil.ch/index.php/altendo

## ETAPE 3 : ENTRER DANS LA NOTICE EN MODE EDITION

In [4]:
def open_autorites(csv_path):
    """
    Charge un fichier CSV, ajoute le suffixe "/edit#identityArea" à toutes les valeurs de la colonne "url_phoebus",
    puis extrait les noms modifiés de la colonne "url_phoebus".

    Args:
        csv_path (str): Chemin du fichier CSV.
    Returns:
        list: Liste des URLs renvoyant à l'édition des notices d'autorité, ouverte sur la Zone d'identification.
    """

    # Charger le fichier CSV
    df = pd.read_csv(csv_path, delimiter=',') #delimiter dépend de la configuration du separator du csv

    # Extraire les noms de la colonne "url_phoebus" et ajouter le suffixe "/edit#identityArea" à toutes les valeurs de la colonne "url_phoebus" 
    df['url_phoebus'] = df['url_phoebus'].astype(str) + '/edit#identityArea'

    # Extraire les noms de la colonne "notice 1" modifiée
    autorites_url = df['url_phoebus'].tolist()
    
    return autorites_url

# LANCEMENT DE LA FONCTION
csv_path = '01_noticesACompleter.csv'
autorites_url = open_autorites(csv_path)
print(autorites_url)

['https://atom-archives.unil.ch/index.php/abriquet-evelyne/edit#identityArea', 'https://atom-archives.unil.ch/index.php/abriquet-michele/edit#identityArea', 'https://atom-archives.unil.ch/index.php/achtoni-marc/edit#identityArea', 'https://atom-archives.unil.ch/index.php/aebischer-jean-marie/edit#identityArea', 'https://atom-archives.unil.ch/index.php/aegata-isabelle/edit#identityArea', 'https://atom-archives.unil.ch/index.php/aguet-pierre/edit#identityArea', 'https://atom-archives.unil.ch/index.php/alamhas-emmanuelle/edit#identityArea', 'https://atom-archives.unil.ch/index.php/alff-wilhelm/edit#identityArea', 'https://atom-archives.unil.ch/index.php/allaz-bernard/edit#identityArea', 'https://atom-archives.unil.ch/index.php/altendorf-wolfgang/edit#identityArea', 'https://atom-archives.unil.ch/index.php/altwegg-jurg/edit#identityArea', 'https://atom-archives.unil.ch/index.php/amaudruz-gaston-armand/edit#identityArea', 'https://atom-archives.unil.ch/index.php/ambuhl-jacques/edit#identity

## ETAPE 4 : OUVRIR LES ZONES ET SAISIR LES INFORMATIONS

In [5]:
def modify_autorites (driver, autorities, autorites_url):
    """
    MOdification et correction en série, ajout de la mention crowdsourcing"

    Cette fonction prend un objet 'driver' pour la navigation web automatisée, une liste d'autorités
    récupérées depuis IdRef, et une liste d'URLs pour les notices d'autorité. Elle ouvre chaque notice d'autorité
    avec le navigateur automatisé, ouvre toutes les Zones, saisit les informations appropriées dans les champs correspondants,
    et sauvegarde les modifications.

    Args:
        driver: Objet pour la navigation web automatisée.
        autorities (list): Liste d'autorités récupérées depuis IdRef.
        autorites_url (list): Liste d'URLs pour les notices d'autorité.

    Returns:
        None
    Notes :
        La fonction inclut les modifications automatiques effectuées par Botorité.
    """   
    try:
        
        for i, autorite_url in enumerate(autorites_url) :
            
            driver.go_to(autorite_url)

            # Ouvrir toutes les Zones
            driver.click('Zone du contrôle')   
            driver.execute_script("window.scrollTo(0,0);")
            time.sleep(2)
            driver.click('Zone de description')
            
            # Récupérer le contenu HTML depuis la page
            content = driver.get_page_source()
            soup = BeautifulSoup(content, features='lxml')
            time.sleep(2)

                
            ## ZONE DESCRIPTION

            # Modification du type d'entité
            driver.click(tag='select', id='entityType')
            driver.click('Personne', tag='option')

            #Date
            if driver.exists(id='datesOfExistence'):
                 # Trouver l'élément par son ID à l'aide de BeautifulSoup
                existence_element = soup.find(id='datesOfExistence')
                if existence_element:
                    existence_value = existence_element.get('value', '')
                    # Vérifier si la valeur est vide ou "n.c."
                    if existence_value == "" :
                        # Si la valeur est vide ou "n.c.", saisir "s.d."
                        driver.type("s.d.", id='datesOfExistence')
                    else:
                        # Sinon, saisir la valeur existante
                        driver.type(existence_value, id='datesOfExistence')
            else:
                print(f"ERREUR pour {autorities[i][1]} : Dates d'existence n'ont pas été trouvées et saisies")

            #Historique
            
            crowdsourcing_value = "**Production participative [*crowdsourcing*]** : Si vous possédez des informations sur cette personne, notamment les dates de naissance et de décès, ainsi que des éléments biographiques, veuillez nous contacter à [clsr@unil.ch](mailto:clsr@unil.ch?subject=[crowdsourcing])."
            if driver.exists(id='history'):
                
                history_element = soup.find(id='history')
                if history_element:
                    
                    history_value = history_element.get('value','')
                    
                    if history_value == "":
                        driver.type(crowdsourcing_value, id='history')
                    else:
                        driver.type(history_value, id='history')
            else:
                print(f"ERREUR pour {autorities[i][1]} : Historique n'a pas été trouvée et saisie")  
                                        
            time.sleep(1)
            
            ## ZONE DE CONTROLE

            # saisie de l'identifiant basé sur l'url
            
            identifiant_url = autorites_url[i]
            identifiant_url = identifiant_url.replace("https://atom-archives.unil.ch/index.php/", "").replace("/edit#identityArea", "")
        
            if driver.exists(id='descriptionIdentifier'):
                
                driver.type(identifiant_url, id='descriptionIdentifier')
            else:
                print(f"ERREUR pour {autorities[i][1]} : Identifiant de la description n'a pas été trouvée et saisie")
            
            # Modification de l'entretien du dépôt
            if driver.exists(id='maintainingRepository'):
                driver.type('Centre des littératures en Suisse romande', id='maintainingRepository')
                time.sleep(4)
                driver.press(driver.Key.ENTER)
            else:
                print(f"ERREUR pour {autorities[i][1]} : Entretien du dépôt de la description n'a pas été trouvé et saisi")
            
                       
            # Modification du statut
            driver.click(tag='select', id='descriptionStatus')
            driver.click('Ébauche', tag='option')
            
            # Notes de maintenance            
            today_date = datetime.today().strftime('%Y-%m-%d à %H:%M:%S')
            maintenance_text = f"""Saisie automatique [L04] effectuée le : """ + today_date
            if driver.exists(id='maintenanceNotes'):
                driver.type(maintenance_text, id='maintenanceNotes')
            else:
                print(f"ERREUR pour {autorities[i][1]} : Notes de maintenance n'ont pas été trouvées et saisies")
      
                
            # scroller vers le bas puis, sauvegarder la saisie
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            time.sleep(2)

            driver.click('Sauvegarder', tag='input')
            
            print(f"({i+1}) Opération de saisie terminée pour {autorities[i][1]} : {autorities[i][0]}")
        
    except Exception as e:
        print(f"Etape 9 : Erreur lors de la modification des notices d'autorité : {e}")
        
# LANCEMENT DE LA FONCTION        
modify_autorites(driver, autorities, autorites_url)


(1) Opération de saisie terminée pour 1 : ABRIQUET Evelyne
(2) Opération de saisie terminée pour 1 : ABRIQUET Michèle
(3) Opération de saisie terminée pour 1 : ACHTONI Marc
(4) Opération de saisie terminée pour 1 : AEBISCHER Jean-Marie
(5) Opération de saisie terminée pour 1 : AEGATA Isabelle
(6) Opération de saisie terminée pour 1 : AGUET Pierre
(7) Opération de saisie terminée pour 1 : ALAMHAS Emmanuelle
(8) Opération de saisie terminée pour 1 : ALFF Wilhelm
(9) Opération de saisie terminée pour 1 : ALLAZ Bernard
(10) Opération de saisie terminée pour 1 : ALTENDORF Wolfgang
(11) Opération de saisie terminée pour 1 : ALTWEGG Jurg
(12) Opération de saisie terminée pour 1 : AMAUDRUZ Gaston-Armand
(13) Opération de saisie terminée pour 1 : AMBÜHL Jacques
(14) Opération de saisie terminée pour 1 : AMSTUTZ Betty
(15) Opération de saisie terminée pour 1 : AMSTUTZ Patrick
(16) Opération de saisie terminée pour 1 : AMY Laurence
(17) Opération de saisie terminée pour 1 : ANCESCHI Luciano
(18) 

## LANCEMENT DES FONCTIONS

In [ ]:
# ETAPE 1
driver = Browser()
access_atom(driver)

# ETAPE 2
csv_path = '01_noticesACompleter.csv'
autorities = extract_autorite(csv_path)

# ETAPE 3
csv_path = '01_noticesACompleter.csv'
autorites_url = open_autorites(csv_path)

# ETAPE 4
modify_autorites(driver, autorities, autorites_url)